### Análise de Polaridade
(comentários fb)

#### 1. <span style="color:#00b3b3">Instalação de pacotes</span>

In [ ]:
if(!require('dplyr')) install.packages('dplyr')
if(!require('purr')) install.packages('purr')
if(!require('tidytext')) install.packages('tidytext')
if(!require('ggplot2')) install.packages('ggplot2')
if(!require('lexiconPT')) install.packages('lexiconPT')
if(!require('wordcloud')) install.packages('wordcloud')

#### 2. <span style="color:#00b3b3">Bibliotecas</span>

In [ ]:
library(dplyr)
library(purrr)
library(tidytext)
library(gutenbergr)
library(ggplot2)
library(lexiconPT)

library(readr)
library(stringr)
library(tidyr)
library(reshape2)
library(syuzhet)
library(ggthemes)
library(wordcloud)

#### 3. <span style="color:#00b3b3">Carrega Dados</span>

In [ ]:
comments <- 

#### 4. <span style="color:#00b3b3">Pré-processamento</span>

In [ ]:
tidy_comments <- comments %>%
  unnest_tokens(word, text) %>%
  anti_join(custom_stop_words)

In [ ]:
custom_stop_words <- bind_rows(stop_words,
                               tibble(word = tm::stopwords("portuguese"),
                                          lexicon = "custom"))
custom_stop_words

**4.** Polaridade

In [ ]:
ggplot(data = pos_neg, aes(x = linenumber, y = value, fill = sentiment)) +
        geom_bar(stat = 'identity', position = position_dodge()) + theme_minimal() +
        ylab("Sentiment") + 
        ggtitle("Positive and Negative Sentiment") +
  scale_color_manual(values = c("darkcyan", "brown3")) +
  scale_fill_manual(values = c("darkcyan", "brown3"))